# Imports

In [1]:
import json
import os

import numpy as np

import pandas as pd

import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader, TensorDataset

# Getting data

In [2]:
train_dataset = pd.read_csv('../datasets/classification_data/classification_train.csv')
test_dataset = pd.read_csv('../datasets/classification_data/classification_test.csv')

### Shuffle data

In [3]:
train_dataset = train_dataset.sample(frac=1).reset_index(drop=True)
test_dataset = test_dataset.sample(frac=1).reset_index(drop=True)

In [4]:
train_dataset.head(5)

,x1,x2,target
0,-0.070726,0.581557,0
1,0.274437,-0.771944,1
2,-0.695878,1.019199,0
3,1.891098,-0.529374,1
4,0.861008,0.519380,0


### Looking for outliers or inconsistencies

In [5]:
train_dataset.describe()

,x1,x2,target
count,670.000000,670.000000,670.000000
mean,0.481824,0.262611,0.505970
std,0.914772,0.615023,0.500338
min,-1.939767,-1.313970,0.000000
25%,-0.169615,-0.159189,0.000000
50%,0.492489,0.275538,1.000000
75%,1.159680,0.717589,1.000000
max,2.590410,1.904169,1.000000


In [6]:
test_dataset.describe()

,x1,x2,target
count,330.000000,330.000000,330.000000
mean,0.472470,0.266104,0.487879
std,0.986760,0.583819,0.500612
min,-1.693028,-1.031435,0.000000
25%,-0.264152,-0.190844,0.000000
50%,0.503077,0.240346,0.000000
75%,1.218472,0.680558,1.000000
max,2.528373,1.783693,1.000000


### Split validation dataset

In [7]:
split_size = int(len(train_dataset) * 0.1)
val_dataset = train_dataset.iloc[:split_size, :]
train_dataset = train_dataset.iloc[split_size:, :]

### Setting dataloader

In [8]:
def get_dataloader(df, batch_size=10, shuffle=True):
    target = torch.tensor(df['target'].values.astype(np.float32))
    data = torch.tensor(df.drop('target', axis = 1).values.astype(np.float32)) 
    tensor = TensorDataset(data, target) 
    return DataLoader(dataset = tensor, batch_size = batch_size, shuffle = True)

# Modeling

In [9]:
class Net(nn.Module):
    def __init__(self, hidden_layer_size=256):
        super(Net, self).__init__()
        self.hid1 = nn.Linear(2, hidden_layer_size)
        self.hid2 = nn.Linear(hidden_layer_size, hidden_layer_size)
        self.oupt = nn.Linear(hidden_layer_size, 1)

        nn.init.xavier_uniform_(self.hid1.weight) 
        nn.init.zeros_(self.hid1.bias)
        nn.init.xavier_uniform_(self.hid2.weight) 
        nn.init.zeros_(self.hid2.bias)
        nn.init.xavier_uniform_(self.oupt.weight)  
        nn.init.zeros_(self.oupt.bias)

    def forward(self, x):
        x = torch.tanh(self.hid1(x)) 
        x = torch.tanh(self.hid2(x))
        x = torch.sigmoid(self.oupt(x)) 
        return x

In [10]:
def __log__(epoch, lr, train_loss, val_loss=None):
    print("epoch {}".format(epoch))
    print("\ttraining loss:    {}".format(train_loss))
    if val_loss is not None:
        print("\tvalidation loss : {}".format(val_loss))
    print("\tlearning rate:    {}".format(lr))

class Model():
    def __init__(self):
        self.network = Net()
    
    def train(self, dataloader, val_dataloader=None, epochs=10, lr=0.001, log_epochs=10, patience=None):
        # setup learning rate
        # weight decay is an option so there must be 2 values
        if isinstance(lr, (list, tuple)):
            lr_bounds = lr[:2]
        else:
            lr_bounds = (lr, lr)
        lr = lr_bounds[0]
        
        # calculate number of steps(dataset length * number of epochs)
        steps = len(dataloader) * epochs
        
        # setup optimizer and loss function
        optimizer = torch.optim.SGD(self.network.parameters(), lr=lr)
        loss_function = nn.BCELoss()
        
        # keep loss so that it can be used to activate an early-stop mechanism(if patience is not None)
        val_losses = []

        for i in range(epochs):
            running_loss = 0.0
            running_val_loss = 0.0
            
            for x, y in dataloader:
                # zero param gradients
                optimizer.zero_grad()
                
                # calculate output and loss
                output = self.network(x)
                loss = loss_function(output, y.reshape(-1,1))
                
                # back-propagate
                loss.backward()
                optimizer.step()
                
                running_loss += loss.item()
                
                # weight decay
                for g in optimizer.param_groups:
                    g['lr'] -= ((lr_bounds[0] - lr_bounds[1]) / steps)
                    lr -= ((lr_bounds[0] - lr_bounds[1]) / steps)

            if val_dataloader is None:
                __log__(i, lr, running_loss / len(train_dataloader))
                continue
                
            with torch.no_grad():
                for x, y in val_dataloader:
                    # zero param gradients
                    optimizer.zero_grad()

                    # calculate output and loss
                    output = self.network(x)
                    running_val_loss += loss_function(output, y.reshape(-1,1))
            __log__(i, lr, running_loss / len(train_dataloader), running_val_loss / len(val_dataloader))
            
            if patience is None:
                continue
                
            val_losses.append(running_val_loss)

            if len(val_losses) < patience + 1:
                continue
            val_losses = val_losses[-(patience + 1):]
            if val_losses.index(min(val_losses)) == 0:
                print('\nEARLY STOP')
                break


    def predict(self, dataloader):        
        predictions = []
        labels = []
        with torch.no_grad():
            for x, y in dataloader:
                # calculate output
                output = self.network(x)
                predicted = self.network(torch.tensor(x, dtype=torch.float32))

                predictions += predicted.reshape(-1).detach().numpy().round().tolist()
                labels += y

        return predictions, labels
    
    def test(self, dataloader):
        predictions, labels = self.predict(dataloader)
    
        counter = {
            'tp': 0, 'fp': 0, 'tn': 0, 'fn': 0
        }
                
        for pred, lbl in zip(predictions, labels):
            if pred == 1 and lbl == 1: res = 'tp'
            elif pred == 1 and lbl == 0: res = 'fp'
            elif pred == 0 and lbl == 0: res = 'tn'
            else: res = 'fn'
            counter[res] += 1

        results = {
            'precision': counter['tp'] / (counter['tp'] + counter['fp']),
            'recall': counter['tp'] / (counter['tp'] + counter['fn'])
        }
        results['f1_score'] = \
            (2 * results['precision'] * results['recall']) / \
            (results['precision'] + results['recall'])
        
        return results
    
    def save(self, dir_path):
        torch.save(
            self.network.state_dict(),
            os.path.join(dir_path, 'weights')
        )


# Training

In [11]:
epochs = 200
hidden_layer_size = 256
batch_size = 10
learning_rate = 0.01
log_epochs = 1

train_dataloader = get_dataloader(train_dataset, batch_size=batch_size)
val_dataloader = get_dataloader(val_dataset, batch_size=batch_size)
test_dataloader = get_dataloader(test_dataset, batch_size=batch_size)

model = Model()
model.train(
    train_dataloader, val_dataloader=val_dataloader,
    epochs=epochs, lr=learning_rate, log_epochs=log_epochs,
    patience=int(epochs*0.1)
)

epoch 0
	training loss:    0.5714566824866123
	validation loss : 0.48037999868392944
	learning rate:    0.01
epoch 1
	training loss:    0.4472302147110955
	validation loss : 0.40821829438209534
	learning rate:    0.01
epoch 2
	training loss:    0.40778477978510935
	validation loss : 0.38177207112312317
	learning rate:    0.01
epoch 3
	training loss:    0.3861849460689748
	validation loss : 0.3567824959754944
	learning rate:    0.01
epoch 4
	training loss:    0.38060415157529176
	validation loss : 0.3739399015903473
	learning rate:    0.01
epoch 5
	training loss:    0.3698070940912747
	validation loss : 0.34478244185447693
	learning rate:    0.01
epoch 6
	training loss:    0.36560543461656964
	validation loss : 0.3363526165485382
	learning rate:    0.01
epoch 7
	training loss:    0.36545764325095004
	validation loss : 0.34436944127082825
	learning rate:    0.01
epoch 8
	training loss:    0.36281279535567174
	validation loss : 0.33262521028518677
	learning rate:    0.01
epoch 9
	training

epoch 75
	training loss:    0.360462555753403
	validation loss : 0.34527334570884705
	learning rate:    0.01
epoch 76
	training loss:    0.3598226620281329
	validation loss : 0.32853227853775024
	learning rate:    0.01
epoch 77
	training loss:    0.3557408431880787
	validation loss : 0.3282530605792999
	learning rate:    0.01
epoch 78
	training loss:    0.356895456915019
	validation loss : 0.3443169593811035
	learning rate:    0.01
epoch 79
	training loss:    0.3565399064148059
	validation loss : 0.33597514033317566
	learning rate:    0.01
epoch 80
	training loss:    0.35740379915862786
	validation loss : 0.3450572192668915
	learning rate:    0.01
epoch 81
	training loss:    0.3582492506162065
	validation loss : 0.3414069712162018
	learning rate:    0.01

EARLY STOP


# Testing

In [12]:
results = model.test(test_dataloader)
results

/Users/joaomacedo/opt/anaconda3/envs/oncase/lib/python3.7/site-packages/ipykernel_launcher.py:88: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


{'precision': 0.8165680473372781,
 'recall': 0.8571428571428571,
 'f1_score': 0.8363636363636363}

In [14]:
path = os.path.join('..', 'models', 'classification')

os.makedirs(path, exist_ok=True)

model.save(path)

with open(os.path.join(path, 'results.json'), 'w') as f:
    f.write(json.dumps(results, indent=4, ensure_ascii=False))

with open(os.path.join(path, 'train_info.json'), 'w') as f:
    f.write(json.dumps({
        'epochs': epochs,
        'batch_size': batch_size,
        'learning_rate': learning_rate,
        'hidden_layer_size': hidden_layer_size
    }))